In [2]:
import sys
import os
import importlib
import pytest # Import pytest for running tests
# Import necessary components from your email_app package
from email_app.new.factory import email_provider_factory
from email_app import settings # This imports settings initially
from email_app import errors

# --- Step 1: Add the project root to Python's path ---
# This is crucial so Jupyter can find the 'email_app' package and pytest can find it too.
project_root = os.path.abspath(os.path.join(os.getcwd()))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print("Email application components loaded.")
print(f"Project root added to sys.path: {project_root}")


# --- Step 2: Configuration for Demo and Test Runs ---
# Set this to 'False' if you want this demo script to attempt real API calls.
# Set to 'True' to force usage of fake APIs for all calls in this notebook.
FORCE_FAKE_APIS_FOR_THIS_NOTEBOOK = False # <--- ADJUST THIS TO True/False

# Set environment variables for this notebook session.
# These will be read by settings.py and the provider classes.
if not FORCE_FAKE_APIS_FOR_THIS_NOTEBOOK:
    # --- IMPORTANT: REPLACE WITH YOUR ACTUAL API KEYS AND DOMAIN ---
    # These will set environment variables for the current Jupyter session.
    # NEVER hardcode real API keys in version control.
    os.environ["SENDGRID_API_KEY"] = "SG.YOUR_ACTUAL_SENDGRID_API_KEY"
    os.environ["MAILGUN_API_KEY"] = "MG.YOUR_ACTUAL_MAILGUN_API_KEY"
    os.environ["MAILGUN_DOMAIN"] = "your_mailgun_sending_domain.com" # e.g., sandbox.mailgun.org
    os.environ["MAILGUN_API_BASE_URL"] = "https://api.mailgun.net/v3/" # or https://api.eu.mailgun.net/v3/

    # Instruct providers to use REAL APIs
    os.environ["USE_FAKE_API_SENDGRID"] = "false"
    os.environ["USE_FAKE_API_MAILGUN"] = "false"

    # --- CRUCIAL: Reload the settings module ---
    # This forces settings.py to re-read the environment variables you just set.
    importlib.reload(settings)

    print("\nEnvironment variables for REAL APIs configured for this demo.")
    print(">>> REMINDER: Make sure your actual API keys and domain are correctly inserted above! <<<")
else:
    # Instruct providers to use FAKE APIs
    os.environ["USE_FAKE_API_SENDGRID"] = "true"
    os.environ["USE_FAKE_API_MAILGUN"] = "true"
    print("\nEnvironment variables configured to use FAKE APIs for this demo.")


# --- Step 3: Demo Usage Function ---
def send_demo_email(provider_name, to_email, subject, content_text=None, content_html=None):
    """Helper function to demonstrate sending an email and print results."""
    print(f"\n--- Attempting to send email via {provider_name} ---")
    try:
        provider = email_provider_factory(provider_name)
        
        provider.set_to_email(to_email).set_subject(subject)
        if content_text:
            provider.set_text_content(content_text)
        elif content_html:
            provider.set_html_content(content_html)
        
        response = provider.send()
        print(f"SUCCESS! {provider_name} Response: {response}")
    except Exception as e:
        print(f"ERROR! Failed to send email via {provider_name}: {e}")

# --- Step 4: Execute Demo Email Sends ---

# Example A: Send via SendGrid with plain text (SUCCESS)
send_demo_email(
    'sendgrid',
    'your_recipient_email@example.com', # <--- CHANGE THIS TO A REAL EMAIL YOU CAN CHECK!
    'Hello from SendGrid Python App (Plain Text) - 日本語', # Unicode in subject
    content_text='This is a plain text email sent from your refactored Python app. Supports Unicode: こんにちは世界! Éàüöç.' # Unicode in content
)

# Example B: Send via Mailgun with HTML content (SUCCESS)
send_demo_email(
    'mailgun',
    'another_recipient_email@example.com', # <--- CHANGE THIS TO A REAL EMAIL YOU CAN CHECK!
    'Hello from Mailgun Python App (HTML) - العربية', # Unicode in subject
    content_html='<h1>Hi there!</h1><p>This is an <b>HTML email</b> from your app. Supports Unicode: مرحبا بالعالم!</p>' # Unicode in content
)

# Example C: Demonstrate an error - Invalid email format (CLIENT-SIDE VALIDATION ERROR)
send_demo_email(
    'sendgrid',
    'invalid-email-format',
    'Error Test',
    content_text='This should fail due to invalid email format.'
)

# Example D: Demonstrate an error - Content too long (CLIENT-SIDE VALIDATION ERROR)
# Note: Default MAX_TEXT_CONTENT_LENGTH is 10000, so this would require making it shorter in settings.py for error.
send_demo_email(
    'mailgun',
    'test@example.com',
    'Long Content Test',
    content_text='Z' * (settings.MAX_TEXT_CONTENT_LENGTH + 1)
)

# Example E: Demonstrate a simulated SendGrid API error (using fake API path, or real if triggered by key/domain)
send_demo_email(
    'sendgrid',
    'error@example.com', # This email address is hardcoded in fake_api.py to trigger an error
    'Simulated API Error Test (SendGrid)',
    content_text='This should trigger a simulated API error.'
)

# Example F: Demonstrate a simulated Mailgun API error (using fake API path, or real if triggered by key/domain)
send_demo_email(
    'mailgun',
    'error@mailgun.com', # This email address is hardcoded in fake_api.py to trigger an error
    'Simulated API Error Test (Mailgun)',
    content_text='This should trigger a simulated API error for Mailgun.'
)

# --- Step 5: Run Pytest Test Suite ---
# This runs your full suite of unit tests. By default, these will use the fake APIs
# due to the fixture in test_email_app.py and the environment variables set above.
print("\n" + "="*80)
print("--- Running Pytest Test Suite (Unit Tests) ---")
print("="*80)
# The '!' prefix tells Jupyter to execute the command in the shell.
# This runs all tests discovered in 'test_email_app.py'.
!pytest test_email_app.py

print("\n" + "="*80)
print("Demo and Test Run Finished.")
print("="*80)



Email application components loaded.
Project root added to sys.path: /home/desalegn/EAI

Environment variables for REAL APIs configured for this demo.
>>> REMINDER: Make sure your actual API keys and domain are correctly inserted above! <<<

--- Attempting to send email via sendgrid ---
--- Sending via REAL SendGrid API ---
ERROR! Failed to send email via sendgrid: SendGrid API Request failed: 401 Client Error: Unauthorized for url: https://api.sendgrid.com/v3/mail/send

--- Attempting to send email via mailgun ---
--- Sending via REAL Mailgun API ---
ERROR! Failed to send email via mailgun: Mailgun API Request failed: 401 Client Error: Unauthorized for url: https://api.mailgun.net/v3/your_mailgun_sending_domain.com/messages

--- Attempting to send email via sendgrid ---
ERROR! Failed to send email via sendgrid: Recipient email 'invalid-email-format' is not valid.

--- Attempting to send email via mailgun ---
ERROR! Failed to send email via mailgun: Plain text content too long for Mail